In [473]:
import re

HINDI_NUMBERS = {
    "शून्य": 0,
    "एक": 1,
    "दो": 2,
    "तीन": 3,
    "चार": 4,
    "पाँच": 5,
    "छह": 6,
    "सात": 7,
    "आठ": 8,
    "नौ": 9,
    "दस": 10,
    "ग्यारह": 11,
    "बारह": 12,
    "तेरह": 13,
    "चौदह": 14,
    "पंद्रह": 15,
    "सोलह": 16,
    "सत्रह": 17,
    "अठारह": 18,
    "उन्नीस": 19,
    "बीस": 20,
    "पच्चीस": 25,
    "तीस": 30,
    "चालीस": 40,
    "पचास": 50
}

MULTIPLIERS = {
    "हज़ार": 1_000,
    "हजार": 1_000,
    "लाख": 100_000
}

In [474]:
def extract_age_from_hindi(text):
    # 1. Try digits first
    nums = re.findall(r'\d+', text)
    if nums:
        return int(nums[0])

    # 2. Try Hindi words
    for word, value in HINDI_NUMBERS.items():
        if word in text:
            return value

    return None

In [475]:
def extract_income_from_hindi(text):
    text = text.replace("₹", "").replace(",", "").strip()

    # Direct digits
    nums = re.findall(r'\d+', text)
    if nums:
        return int(nums[0])

    total = 0

    # Special cases
    if "ढाई लाख" in text:
        return 250000
    if "साढ़े दो लाख" in text:
        return 250000

    # Handle "X लाख"
    for word, value in HINDI_NUMBERS.items():
        if word in text and "लाख" in text:
            total += value * 100000

    # Handle "Y हज़ार"
    for word, value in HINDI_NUMBERS.items():
        if word in text and ("हज़ार" in text or "हजार" in text):
            total += value * 1000

    if total > 0:
        return total

    return None


In [476]:
class Memory:
    def __init__(self):
        self.data = {}
        self.pending = None

    def update(self, key, value):
        self.data[key] = value
        self.pending = None

    def get(self, key):
        return self.data.get(key)

    def all(self):
        return self.data

memory = Memory()


In [477]:
def eligibility_tool(memory):
    """
    Simple rule-based eligibility engine.
    Input: memory (age, income)
    Output: list of eligible schemes
    """

    age = memory.get("age")
    income = memory.get("income")

    eligible_schemes = []

    if age is not None and income is not None:
        if age >= 18 and income <= 200000:
            eligible_schemes.append("प्रधानमंत्री आवास योजना")

        if age >=40 and income <= 200000:
            eligible_schemes.append("प्रधानमंत्री उज्ज्वला योजना")

    return eligible_schemes


In [478]:
from ollama import chat

def llm_planner(user_text, memory):
    prompt = f"""
You are an intelligent assistant helping users apply for government schemes.

Conversation memory:
{memory}

User says:
"{user_text}"

Return ONLY valid JSON in this exact format:

{{
  "intent": "provide_age | provide_income | ask_scheme | unknown",
  "action": "store_age | store_income | check_eligibility | ask_clarification"
}}

Do NOT include any explanation or extra text.
"""

    response = chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )

    return response["message"]["content"]


In [479]:
import json

def parse_llm_output(text):
    try:
        parsed = json.loads(text)

        # Ensure required keys exist
        if not isinstance(parsed, dict):
            raise ValueError("Not a dict")

        if "intent" not in parsed or "action" not in parsed:
            raise ValueError("Missing required keys")

        return parsed

    except Exception:
        return {
            "intent": "unknown",
            "action": "ask_clarification"
        }


In [480]:
def executor(plan, text, memory):

    intent = plan.get("intent")

    # -------------------------------------------------
    # 1️⃣ HANDLE CONFIRMATION (YES / NO)
    # -------------------------------------------------
    if memory.pending:
        if any(word in text for word in ["हाँ", "हाँजी", "जी"]):
            field, value = memory.pending
            memory.update(field, value)
            memory.pending = None
            return f"ठीक है, आपकी {field} अब {value} के रूप में अपडेट कर दी गई है।"

        if any(word in text for word in ["नहीं", "ना"]):
            memory.pending = None
            return "ठीक है, परिवर्तन रद्द कर दिया गया।"

        return "कृपया 'हाँ' या 'नहीं' में उत्तर दें।"

    # -------------------------------------------------
    # 2️⃣ HANDLE AGE INPUT
    # -------------------------------------------------
    if intent == "provide_age":
        age = extract_age_from_hindi(text)
        if age is None:
            return "कृपया अपनी उम्र बताएं।"

        prev = memory.get("age")
        if prev is not None and prev != age:
            memory.pending = ("age", age)
            return f"आपने पहले {prev} वर्ष बताया था, अब {age} कह रहे हैं। क्या आप इसे बदलना चाहते हैं?"

        memory.update("age", age)
        return f"ठीक है, आपकी उम्र {age} वर्ष दर्ज कर ली गई है। अब आपकी आय बताएं।"

    # -------------------------------------------------
    # 3️⃣ HANDLE INCOME
    # -------------------------------------------------
    if intent == "provide_income":
        income = extract_income_from_hindi(text)
        if income is None:
            return "कृपया अपनी आय बताएं।"

        prev = memory.get("income")
        if prev is not None and prev != income:
            memory.pending = ("income", income)
            return f"आपने पहले {prev} रुपये बताए थे, अब {income} कह रहे हैं। क्या आप इसे बदलना चाहते हैं?"

        memory.update("income", income)

        schemes = eligibility_tool(memory)
        if schemes:
            return f"आप इन योजनाओं के लिए पात्र हैं: " + " और ".join(schemes)
        else:
            return "आप किसी भी योजना के लिए पात्र नहीं हैं।"

    # -------------------------------------------------
    # 4️⃣ HANDLE GENERAL / UNKNOWN
    # -------------------------------------------------
    return "कृपया अपनी उम्र और आय बताएं, ताकि मैं आपकी मदद कर सकूँ।"


In [482]:
from gtts import gTTS
from IPython.display import Audio, display
import uuid
import os

def speak_hindi(text):
    """
    Reliable Hindi TTS using gTTS.
    """
    filename = f"tts_{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=text, lang="hi")
    tts.save(filename)

    display(Audio(filename, autoplay=True))

    # Optional cleanup
    # os.remove(filename)

In [483]:
def run_agent_turn(memory):
    # 1. Record voice
    import sounddevice as sd
    import soundfile as sf

    fs = 16000
    duration = 5

    print("Speak now...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="int16")
    sd.wait()
    sf.write("input.wav", audio, fs, subtype="PCM_16")

    # 2. STT (Vosk)
    from vosk import Model, KaldiRecognizer
    import json

    model = Model("models/vosk-hi")
    data, samplerate = sf.read("input.wav", dtype="int16")

    rec = KaldiRecognizer(model, samplerate)
    rec.AcceptWaveform(data.tobytes())
    result = json.loads(rec.FinalResult())
    user_text = result.get("text", "")

    print("User said:", user_text)

    # 3. Agent reasoning
    raw = llm_planner(user_text, memory)
    plan = parse_llm_output(raw)
    response = executor(plan, user_text, memory)

    # 4. Output
    print("Agent:", response)
    if response and isinstance(response, str):
        speak_hindi(response)
    else:
        print("No valid response to speak.")
    print("Memory:", memory.all())


In [484]:
def run_conversation():
    global memory
    print("🎙️ Voice assistant started. Say 'exit' to stop.\n")

    while True:
        run_agent_turn(memory)
        
        cont = input("\nPress ENTER to continue or type 'exit' to stop: ")

        if cont.lower().strip() == "exit":
            print("Conversation ended.")
            memory = Memory()   # RESET MEMORY
            break

In [485]:
run_conversation()


🎙️ Voice assistant started. Say 'exit' to stop.

Speak now...
User said: मुझे सरकारी योजना के लिए आवेदन करना है
Agent: कृपया अपनी उम्र या आय बताएं ताकि मैं आपकी सहायता कर सकूँ।


Memory: {}



Press ENTER to continue or type 'exit' to stop:  


Speak now...
User said: मेरी उम्र पच्चीस साल है
Agent: ठीक है, आपकी उम्र 25 वर्ष दर्ज कर ली गई है। अब आपकी आय बताएं।


Memory: {'age': 25}


...


Press ENTER to continue or type 'exit' to stop:  


Speak now...
User said: मेरी आय दो लाख है
Agent: आप इन योजनाओं के लिए पात्र हैं: प्रधानमंत्री आवास योजना


Memory: {'age': 25, 'income': 200000}



Press ENTER to continue or type 'exit' to stop:  exit


Conversation ended.
